In [90]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

EPOCHS = 10
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 43
TEST_SIZE = 0.4


def main():

    # Check command-line arguments
    if len(sys.argv) not in [2, 3]:
        sys.exit("Usage: python traffic.py data_directory [model.h5]")

    # Get image arrays and labels for all image files
    images, labels = load_data(sys.argv[1])

    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )

    # Get a compiled neural network
    model = get_model()

    # Fit model on training data
    model.fit(x_train, y_train, epochs=EPOCHS)

    # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=2)

    # Save model to file
    if len(sys.argv) == 3:
        filename = sys.argv[2]
        model.save(filename)
        print(f"Model saved to {filename}.")


def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = []
    labels = []
    
    for path, _, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.ppm'):
                
                image = cv2.imread(os.path.join(path, file))
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                
                # add to list
                images.append(image)
                labels.append(os.path.basename(path))
    
    return images, labels

def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    model = tf.keras.Sequential(name="my_test_1")
    model.add(keras.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))) 
    
    # 1 layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)))
    model.add(keras.layers.MaxPool2D(3, 3))
    
    # 2 layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPool2D(3, 3))
    
    # flatern 
    model.add(keras.layers.Flatten())
    
    # Hiden with drop
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    
    # out
    model.add(keras.layers.Dense(NUM_CATEGORIES, activation='softmax'))
    
    print(model.summary())
    
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model
    

# if __name__ == "__main__":
#     main()

images, labels = load_data('/home/v/Downloads/cs50_2020/traffic/gtsrb/0')

get_model()

Model: "my_test_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 28, 28, 32)        896       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 9, 9, 32)          0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 7, 7, 32)          9248      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 2, 2, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0 